### How to use keras Tuner in our ANN model
#### We use keras Tuner to select some of the best parameter to make a perfect or good model.
* Lets See how to us it!!!
#### First, We import libraries and divide our data into test and train split.

In [1]:
import pandas as pd
dataset = pd.read_csv('Churn_Modelling.csv') # Importing the dataset
X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13]

geography=pd.get_dummies(X["Geography"],drop_first=True) # Implement one-hot encoding
gender=pd.get_dummies(X['Gender'],drop_first=True)


X=pd.concat([X,geography,gender],axis=1)## Concatenate the Data Frames


X=X.drop(['Geography','Gender'],axis=1)## Drop Unnecessary columns

from sklearn.model_selection import train_test_split # Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


In [2]:
X_train.shape

(8000, 11)

#### Now We make a function and give some parameter to check.Which parameter perform good .
#### For more detail information .Kindly refer Keras Tunner official documantaion. Click Here !!!
* https://keras-team.github.io/keras-tuner/

In [3]:
def build_model(hp):        
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

In [4]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
tuner = RandomSearch(                      # To fit our model 
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='project1',
    project_name='Churn')

In [5]:
tuner.search_space_summary() #To check the su

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


### Here we want to go for 5 trails(5 epochs)

In [6]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

Trial 5 Complete [00h 01m 13s]
val_accuracy: 0.7975000143051147

Best val_accuracy So Far: 0.7975000143051147
Total elapsed time: 00h 04m 26s
INFO:tensorflow:Oracle triggered exit


### Here we check the best parameter 

In [7]:
tuner.get_best_hyperparameters

<bound method BaseTuner.get_best_hyperparameters of <kerastuner.tuners.randomsearch.RandomSearch object at 0x00000154CAE1E240>>

In [12]:
models=tuner.get_best_models(num_models=2)

#### Here we check top parmeter which will give you best result.

In [14]:
tuner.results_summary()

Results summary
Results in project1\Churn
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
num_layers: 9
units_0: 448
units_1: 384
learning_rate: 0.001
units_2: 32
units_3: 32
units_4: 32
units_5: 32
units_6: 32
units_7: 32
units_8: 32
Score: 0.7975000143051147
Trial summary
Hyperparameters:
num_layers: 12
units_0: 160
units_1: 96
learning_rate: 0.0001
units_2: 384
units_3: 352
units_4: 384
units_5: 384
units_6: 256
units_7: 352
units_8: 352
units_9: 32
units_10: 32
units_11: 32
Score: 0.7975000143051147
Trial summary
Hyperparameters:
num_layers: 7
units_0: 512
units_1: 384
learning_rate: 0.0001
units_2: 352
units_3: 352
units_4: 448
units_5: 320
units_6: 352
units_7: 224
units_8: 96
units_9: 128
units_10: 320
units_11: 128
Score: 0.7975000143051147
Trial summary
Hyperparameters:
num_layers: 19
units_0: 320
units_1: 448
learning_rate: 0.001
units_2: 64
units_3: 64
units_4: 32
units_5: 128
units_6: 416
units_7: 512
units_8: 192
units_

### Great!!! Now we follow same procedure to make our final model. For prev Kindly refer this notebook.
* https://github.com/AmitSingh-Rajjput/Deep-Learning-Tutorials/blob/main/ANN%20Classification%20Customer%20Churn%20Data.ipynb